In [ ]:
import pandas as pd
import torch
import torchvision
from torchvision.transforms import ToTensor
from pandas import Series
import numpy as np
from sklearn import preprocessing

In [ ]:
#The dataset, collected on the Marconi 100 system, has been published and
#described in the Scientific dataset publication: https://www.nature.com/articles/s41597-023-02174-3
#It is available on Zenodo: https://zenodo.org/records/7541722


#This is rack 1 of the dataset
!wget https://zenodo.org/record/7541722/files/1.tar?download=1 -o rack.tar

In [ ]:
!tar -xf /content/1.tar?download=1

In [ ]:
#This is node 36, of rack 1
file_name = '36.parquet'

In [ ]:
DATA = pd.read_parquet(file_name)
DATA.head(5)

,timestamp,ambient_avg,ambient_std,ambient_min,ambient_max,dimm0_temp_avg,dimm0_temp_std,dimm0_temp_min,dimm0_temp_max,dimm10_temp_avg,...,ps1_output_curre_max,ps1_output_volta_avg,ps1_output_volta_std,ps1_output_volta_min,ps1_output_volta_max,total_power_avg,total_power_std,total_power_min,total_power_max,value
0,2020-03-09 12:00:00+00:00,21.636362,0.077138,21.600000,21.799999,29.0,0.0,29.0,29.0,30.0,...,22.0,12.399999,0.0,12.4,12.4,400.0,0.0,400.0,400.0,<NA>
1,2020-03-09 12:15:00+00:00,21.860000,0.128063,21.600000,22.000000,29.0,0.0,29.0,29.0,30.0,...,21.0,12.399998,0.0,12.4,12.4,400.0,0.0,400.0,400.0,<NA>
2,2020-03-09 12:30:00+00:00,21.885714,0.098975,21.799999,22.000000,29.0,0.0,29.0,29.0,30.0,...,21.0,12.399999,0.0,12.4,12.4,400.0,0.0,400.0,400.0,<NA>
3,2020-03-09 12:45:00+00:00,21.918917,0.098194,21.799999,22.000000,29.0,0.0,29.0,29.0,30.0,...,20.0,12.399999,0.0,12.4,12.4,400.0,0.0,400.0,400.0,<NA>
4,2020-03-09 13:00:00+00:00,21.875553,0.096967,21.799999,22.000000,29.0,0.0,29.0,29.0,30.0,...,21.0,12.400001,0.0,12.4,12.4,400.0,0.0,400.0,400.0,<NA>


In [ ]:
#The dataset has periods of unavailability of Nagios traces, as described in the dataset paper:  https://www.nature.com/articles/s41597-023-02174-3/figures/3
#We suggest dropping the periods where the labels are largely unavailable and using the dataset either after 1.4.2021 or 1.10.2021.
DATA = DATA[DATA['timestamp'] > '2021-04-01']

### parisa change
DATA['value'] = DATA['value'].replace(2,1)
DATA['value'] = DATA['value'].replace(3,1)
## parisa change

In [ ]:
DATA.reset_index(drop=True, inplace = True)
DATA = DATA.fillna(0)
DATA['timestamp'] = pd.to_datetime(DATA['timestamp'])
l = DATA.timestamp.diff() == pd.Timedelta(minutes=15) #time consistency -> measurements should be 15 minutes apart
chunks = []
current_chunk = []
for index, value in enumerate(l):
    current_chunk.append(DATA.iloc[index])
    if not value:
        chunks.append(pd.DataFrame(current_chunk))
        current_chunk = []


if current_chunk:
    chunks.append(pd.DataFrame(current_chunk))

In [ ]:
DATA['value'] = DATA['value'].replace(2,1)
DATA['value'] = DATA['value'].replace(3,1)

### parisa change
relevant = [c for c in chunks if len(c) >= 20] #assuming time window of 20
revelent_len = [len(c) for c in chunks if len(c) >= 20]
DATA2 = pd.concat(relevant)
DATA = DATA2
DATA['value'] = DATA['value'].replace(2,1)
DATA['value'] = DATA['value'].replace(3,1)

### parisa change end
##3 parisa change
# relevant = [c for c in chunks if len(c) >= 20] #assuming time window of 20
# DATA2 = pd.concat(relevant)
# DATA = DATA2
# DATA['value'] = DATA['value'].replace(2,1)
# DATA['value'] = DATA['value'].replace(3,1)
# DATA = DATA.drop(columns=['timestamp'])
# DATA = DATA.astype(float)
##3 parisa change end

In [ ]:
DATA.head(5)

,timestamp,ambient_avg,ambient_std,ambient_min,ambient_max,dimm0_temp_avg,dimm0_temp_std,dimm0_temp_min,dimm0_temp_max,dimm10_temp_avg,...,ps1_output_curre_max,ps1_output_volta_avg,ps1_output_volta_std,ps1_output_volta_min,ps1_output_volta_max,total_power_avg,total_power_std,total_power_min,total_power_max,value
1,2021-04-02 00:45:00+00:00,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0
2,2021-04-02 01:00:00+00:00,24.785712,0.051508,24.6,24.799999,33.0,0.0,33.0,33.0,33.0,...,35.0,12.357142,0.072843,12.2,12.4,742.857143,101.940359,580.0,920.0,0
3,2021-04-02 01:15:00+00:00,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0
4,2021-04-02 01:30:00+00:00,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0
5,2021-04-02 01:45:00+00:00,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0


In [ ]:
DATA3= DATA
DATA=DATA.drop(columns=['timestamp'])
original_dtypes = DATA.dtypes
DATA = DATA.astype(float)

In [ ]:
df_value_0 = DATA[DATA['value'] == 0.0].drop(columns=['value']).to_numpy()

# Separate rows where 'value' is 1.0
df_value_1 = DATA[DATA['value'] == 1.0].drop(columns=['value']).to_numpy()

In [ ]:
### use for delete value column
DATA=DATA.drop(columns=['value'])
original_dtypes = DATA.dtypes

In [ ]:
### use for delete value column
scaler = preprocessing.MinMaxScaler()  ### use further
names = DATA.columns  ##
d_0 = scaler.fit_transform(df_value_0)
d_1= scaler.fit_transform(df_value_1)
df_scale_1 = pd.DataFrame(d_1, columns=names)
df_scale_0 = pd.DataFrame(d_0, columns=names)

In [ ]:
# scaler = preprocessing.MinMaxScaler()  ### use further
# names = DATA.columns  ##
# d = scaler.fit_transform(DATA)
# DATA = pd.DataFrame(d, columns=names)
# ### parisa change                                --- here to change more or add time stamp in train

#timestamp_scaler = preprocessing.MinMaxScaler()
#DATA['timestamp'] = timestamp_scaler.fit_transform(DATA3['timestamp'].astype('int64').values.reshape(-1, 1))
#### parisa change end

In [ ]:
# train_data = DATA[:int(DATA.shape[0]*0.8)]
# test_data = DATA[int(DATA.shape[0]*0.8):]

################# parisa change
# train_data = DATA[:]

############### end parisa change

In [ ]:
df_value_1.shape

(1311, 352)

In [ ]:
df_value_0.shape

(50998, 352)

In [ ]:
train_data['value'].value_counts()

,count
value,
0.0,50998
1.0,1311


In [ ]:
# df_value_0 = df_value_0.drop(columns=['value']).to_numpy()
# # train_feat = train_feat.to_numpy()
# df_value_1 = df_value_1.drop(columns=['value']).to_numpy()
# # test_feat = test_feat.to_numpy()

AttributeError: 'numpy.ndarray' object has no attribute 'drop'

In [ ]:
df_value_0 = torch.tensor(df_scale_0.to_numpy(), dtype=torch.float)
df_value_1 = torch.tensor(df_scale_1.to_numpy(), dtype=torch.float)


In [ ]:
# test_labels = test_data['value']
# test_labels = test_labels.to_numpy()
# test_labels = torch.tensor(test_labels, dtype=torch.float)
# test_labels

tensor([0., 0., 0.,  ..., 0., 0., 0.])

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_window = 20 ## 20

def create_data_seq(data,tw):
    seq = []
    for i in range(len(data)-tw):
        x_seq = data[i:i+tw],
        y_seq = data[i+tw:i+tw+1]  ###در ترین میاد 0 تا 20 رو میده و میگه 21 رو حدس بزن
        seq.append((x_seq,y_seq))
    return seq


In [ ]:
# train_seq = create_data_seq(train_feat,train_window,revelent_len) ## parisa change
train_seq_value_0 = create_data_seq(df_value_0,train_window) ## parisa change
train_seq_value_1 = create_data_seq(df_value_1,train_window) ## parisa change

In [ ]:
X[0].size()


torch.Size([1, 20, 352])

In [ ]:
train_loader_value_0 = DataLoader(train_seq_value_0, batch_size=1, shuffle=True)

X,y  = next(iter(train_loader_value_0))
print(X)


[tensor([[[0.5128, 0.0204, 0.5425,  ..., 0.2262, 0.5405, 0.7500],
         [0.5160, 0.0330, 0.5425,  ..., 0.1918, 0.5270, 0.7396],
         [0.5119, 0.0226, 0.5377,  ..., 0.2343, 0.4865, 0.7396],
         ...,
         [0.5205, 0.0000, 0.5519,  ..., 0.1885, 0.6216, 0.7188],
         [0.5212, 0.0217, 0.5472,  ..., 0.2212, 0.5676, 0.7396],
         [0.5219, 0.0235, 0.5472,  ..., 0.1575, 0.6216, 0.6979]]])]


In [ ]:
train_loader_value_1 = DataLoader(train_seq_value_1, batch_size=1, shuffle=False) ## در هر فور ما یه تعدادی بچ داریم که میدیم به دل با بچ بیشتر میتونیم بگیم مثلا 2 تا 2 تا بهش بده
X,y = next(iter(train_loader_value_1))

In [ ]:
class encoder(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.lstm1 = torch.nn.LSTM(
            input_size=352,
            hidden_size=16,
            batch_first=True,
            num_layers=1
        )
        self.lstm2 = torch.nn.LSTM(
          input_size=16,
          hidden_size=8,
          num_layers=1,
          batch_first=True
        )

    def forward(self, x):
        batch_size = x.shape[0]
        h0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)

        x, (_, _) = self.lstm1(x, (h0, c0))

        h0 = torch.zeros(1, batch_size,8).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
        x, (_, _) = self.lstm2(x, (h0, c0))
        return x[:, :, :]

In [ ]:
############### mine
# class encoder(torch.nn.Module):
#     def __init__(self):
#         super().__init__()

#         self.lstm1 = torch.nn.LSTM(
#             input_size=352,
#             hidden_size=32,
#             batch_first=True,
#             num_layers=1
#         )
#         self.lstm2 = torch.nn.LSTM(
#           input_size=32,
#           hidden_size=8,
#           num_layers=1,
#           batch_first=True
#         )

#     def forward(self, x):
#         batch_size = x.shape[0]
#         h0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
#         c0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)

#         x, (_, _) = self.lstm1(x, (h0, c0))

#         h0 = torch.zeros(1, batch_size,8).requires_grad_().to(device)
#         c0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
#         x, (_, _) = self.lstm2(x, (h0, c0))
#         return x[:, :, :]

class encoder(torch.nn.Module):
    def __init__(self):
        super().__init__()

        # First LSTM layer
        self.lstm1 = torch.nn.LSTM(
            input_size=353,
            hidden_size=32,  # Increased the hidden size for this example
            batch_first=True,
            num_layers=1
        )

        # Second LSTM layer
        self.lstm2 = torch.nn.LSTM(
            input_size=32,   # Input size matches the hidden size of lstm1
            hidden_size=16,
            batch_first=True,
            num_layers=1
        )

        # Third LSTM layer (newly added)
        self.lstm3 = torch.nn.LSTM(
            input_size=16,   # Input size matches the hidden size of lstm2
            hidden_size=8,
            batch_first=True,
            num_layers=1
        )

    def forward(self, x):
        batch_size = x.shape[0]

        # Initial hidden and cell states for lstm1
        h0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        x, (_, _) = self.lstm1(x, (h0, c0))

        # Initial hidden and cell states for lstm2
        h0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        x, (_, _) = self.lstm2(x, (h0, c0))

        # Initial hidden and cell states for lstm3
        h0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
        x, (_, _) = self.lstm3(x, (h0, c0))

        return x[:, :, :]

In [ ]:
class decoder(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.lstm1 = torch.nn.LSTM(
            input_size=8,
            hidden_size=16,
            batch_first=True,
            num_layers=1
        )
        self.lstm2 = torch.nn.LSTM(
          input_size=16,
          hidden_size=352,
          num_layers=1,
          batch_first=True
        )

    def forward(self, x):
        batch_size = x.shape[0]
        h0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)

        x, (_, _) = self.lstm1(x, (h0, c0))

        h0 = torch.zeros(1, batch_size, 352).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 352).requires_grad_().to(device)
        x, (_, _) = self.lstm2(x, (h0, c0))
        return x

In [ ]:
### mine
class decoder(torch.nn.Module):
    def __init__(self):
        super().__init__()

        # First LSTM layer (added for symmetry with the encoder)
        self.lstm1 = torch.nn.LSTM(
            input_size=8,
            hidden_size=16,
            batch_first=True,
            num_layers=1
        )

        # Second LSTM layer
        self.lstm2 = torch.nn.LSTM(
            input_size=16,
            hidden_size=32,
            batch_first=True,
            num_layers=1
        )

        # Third LSTM layer
        self.lstm3 = torch.nn.LSTM(
            input_size=32,
            hidden_size=353,
            batch_first=True,
            num_layers=1
        )

    def forward(self, x):
        batch_size = x.shape[0]

        # Initial hidden and cell states for lstm1
        h0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        x, (_, _) = self.lstm1(x, (h0, c0))

        # Initial hidden and cell states for lstm2
        h0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        x, (_, _) = self.lstm2(x, (h0, c0))

        # Initial hidden and cell states for lstm3
        h0 = torch.zeros(1, batch_size, 353).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 353).requires_grad_().to(device)
        x, (_, _) = self.lstm3(x, (h0, c0))

        return x

In [ ]:
############### mine for add h,c
# class encoder(torch.nn.Module):
#     def __init__(self):
#         super().__init__()

#         self.lstm1 = torch.nn.LSTM(
#             input_size=352,
#             hidden_size=32,
#             batch_first=True,
#             num_layers=1
#         )
#         self.lstm2 = torch.nn.LSTM(
#           input_size=32,
#           hidden_size=8,
#           num_layers=1,
#           batch_first=True
#         )

#     def forward(self, x):
#         batch_size = x.shape[0]
#         h0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
#         c0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)

#         x, (_, _) = self.lstm1(x, (h0, c0))

#         h0 = torch.zeros(1, batch_size,8).requires_grad_().to(device)
#         c0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
#         x, (_, _) = self.lstm2(x, (h0, c0))
#         return x[:, :, :]

class encoder(torch.nn.Module):
    def __init__(self):
        super().__init__()

        # First LSTM layer
        self.lstm1 = torch.nn.LSTM(
            input_size=352,
            hidden_size=32,  # Increased the hidden size for this example
            batch_first=True,
            num_layers=1
        )

        # Second LSTM layer
        self.lstm2 = torch.nn.LSTM(
            input_size=32,   # Input size matches the hidden size of lstm1
            hidden_size=16,
            batch_first=True,
            num_layers=1
        )

        # Third LSTM layer (newly added)
        self.lstm3 = torch.nn.LSTM(
            input_size=16,   # Input size matches the hidden size of lstm2
            hidden_size=8,
            batch_first=True,
            num_layers=1
        )
        self.h_proj_1_to_2 = torch.nn.Linear(32, 16)
        self.c_proj_1_to_2 = torch.nn.Linear(32, 16)

        self.h_proj_2_to_3 = torch.nn.Linear(16, 8)
        self.c_proj_2_to_3 = torch.nn.Linear(16, 8)

    def forward(self, x):
        batch_size = x.shape[0]
        # Initial hidden and cell states for lstm1
        # h0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        # x, (_, _) = self.lstm1(x, (h0, c0))

        # # Initial hidden and cell states for lstm2
        # h0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        # x, (_, _) = self.lstm2(x, (h0, c0))

        # # Initial hidden and cell states for lstm3
        # h0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
        # x, (_, _) = self.lstm3(x, (h0, c0))

        ###### parisa changes of model give h , c to new layaer

        # Initial hidden and cell states for lstm1
        # Initial hidden and cell states for lstm1
        h0 = torch.zeros(1, batch_size, 32).to(x.device)
        c0 = torch.zeros(1, batch_size, 32).to(x.device)
        x, (h1, c1) = self.lstm1(x, (h0, c0))

        # Resize h1, c1 to match the input size of lstm2
        h1 = self.h_proj_1_to_2(h1)
        c1 = self.c_proj_1_to_2(c1)
        x, (h2, c2) = self.lstm2(x, (h1, c1))

        # Resize h2, c2 to match the input size of lstm3
        h2 = self.h_proj_2_to_3(h2)
        c2 = self.c_proj_2_to_3(c2)
        x, (h3, c3) = self.lstm3(x, (h2, c2))
        return x[:, :, :]

In [ ]:
### mine for add h,c
class decoder(torch.nn.Module):
    def __init__(self):
        super().__init__()

        # First LSTM layer (added for symmetry with the encoder)
        self.lstm1 = torch.nn.LSTM(
            input_size=8,
            hidden_size=16,
            batch_first=True,
            num_layers=1
        )

        # Second LSTM layer
        self.lstm2 = torch.nn.LSTM(
            input_size=16,
            hidden_size=32,
            batch_first=True,
            num_layers=1
        )

        # Third LSTM layer
        self.lstm3 = torch.nn.LSTM(
            input_size=32,
            hidden_size=352,
            batch_first=True,
            num_layers=1
        )

        self.h_proj_1_to_2 = torch.nn.Linear(16, 32)
        self.c_proj_1_to_2 = torch.nn.Linear(16, 32)

        self.h_proj_2_to_3 = torch.nn.Linear(32, 352)
        self.c_proj_2_to_3 = torch.nn.Linear(32, 352)

    def forward(self, x):
        batch_size = x.shape[0]

        # # Initial hidden and cell states for lstm1
        # h0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        # x, (_, _) = self.lstm1(x, (h0, c0))

        # # Initial hidden and cell states for lstm2
        # h0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        # x, (_, _) = self.lstm2(x, (h0, c0))

        # # Initial hidden and cell states for lstm3
        # h0 = torch.zeros(1, batch_size, 352).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 352).requires_grad_().to(device)
        # x, (_, _) = self.lstm3(x, (h0, c0))


        ###### parisa changes of model give h , c to new layaer

        # Initial hidden and cell states for lstm1
        h1 = torch.zeros(1, batch_size, 16).to(x.device)
        c1 = torch.zeros(1, batch_size, 16).to(x.device)
        x, (h1, c1) = self.lstm1(x, (h1, c1))

        # Resize h1, c1 to match the input size of lstm2
        h2 = self.h_proj_1_to_2(h1)
        c2 = self.c_proj_1_to_2(c1)
        x, (h2, c2) = self.lstm2(x, (h2, c2))

        # Resize h2, c2 to match the input size of lstm3
        h3 = self.h_proj_2_to_3(h2)
        c3 = self.c_proj_2_to_3(c2)
        x, (h3, c3) = self.lstm3(x, (h3, c3))

        return x

In [ ]:
class AE(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = encoder().to(device)
        self.decoder = decoder().to(device)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
class RUAD(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = encoder().to(device)
        self.fc1 = torch.nn.Linear(8,16)
        self.fc2 = torch.nn.Linear(16,352)


    def forward(self, x):
        x = self.encoder(x)
        #print('LSTM out', x.shape)
        x = self.fc1(x)
        #print('DENSE 1', x.shape)
        x = self.fc2(x)
        #print('DENSE 2', x.shape)

        return x

In [ ]:
#### mine
# class RUAD(torch.nn.Module):
#     def __init__(self):
#         super().__init__()

#         self.encoder = encoder().to(device)
#         self.fc1 = torch.nn.Linear(8,16)
#         self.fc2 = torch.nn.Linear(16,352)


#     def forward(self, x):
#         x = self.encoder(x)
#         #print('LSTM out', x.shape)
#         x = self.fc1(x)
#         #print('DENSE 1', x.shape)
#         x = self.fc2(x)
#         #print('DENSE 2', x.shape)

#         return x

In [ ]:
### mine
class RUAD(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = encoder().to(device)
        self.fc1 = torch.nn.Linear(8, 16)
        self.fc2 = torch.nn.Linear(16, 8)
        self.decoder = decoder().to(device)  # Adding the decoder to RUAD

    def forward(self, x):
        x = self.encoder(x)
        # Passing through the fully connected layers   ----- new changes
        # x = self.fc1(x)
        # x = self.fc2(x)
        # Passing the result through the decoder ------- end of new changes
        x = self.decoder(x)

        return x

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = RUAD()
model

RUAD(
  (encoder): encoder(
    (lstm1): LSTM(352, 32, batch_first=True)
    (lstm2): LSTM(32, 16, batch_first=True)
    (lstm3): LSTM(16, 8, batch_first=True)
    (h_proj_1_to_2): Linear(in_features=32, out_features=16, bias=True)
    (c_proj_1_to_2): Linear(in_features=32, out_features=16, bias=True)
    (h_proj_2_to_3): Linear(in_features=16, out_features=8, bias=True)
    (c_proj_2_to_3): Linear(in_features=16, out_features=8, bias=True)
  )
  (fc1): Linear(in_features=8, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (decoder): decoder(
    (lstm1): LSTM(8, 16, batch_first=True)
    (lstm2): LSTM(16, 32, batch_first=True)
    (lstm3): LSTM(32, 352, batch_first=True)
    (h_proj_1_to_2): Linear(in_features=16, out_features=32, bias=True)
    (c_proj_1_to_2): Linear(in_features=16, out_features=32, bias=True)
    (h_proj_2_to_3): Linear(in_features=32, out_features=352, bias=True)
    (c_proj_2_to_3): Linear(in_features=32, out_features=352,

In [ ]:
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)  ### adam ro bezar va bebin
criterion = torch.nn.MSELoss()

In [ ]:
for d in train_loader_value_0:
    print(len(d[0]))
    print(d[0][0].size())
    print(d[1].size())
    break
    ## batch size,(row count,column count) --> sequence

1
torch.Size([1, 20, 353])
torch.Size([1, 1, 353])


In [ ]:
print(model)

RUAD(
  (encoder): encoder(
    (lstm1): LSTM(353, 32, batch_first=True)
    (lstm2): LSTM(32, 16, batch_first=True)
    (lstm3): LSTM(16, 8, batch_first=True)
    (h_proj_1_to_2): Linear(in_features=32, out_features=16, bias=True)
    (c_proj_1_to_2): Linear(in_features=32, out_features=16, bias=True)
    (h_proj_2_to_3): Linear(in_features=16, out_features=8, bias=True)
    (c_proj_2_to_3): Linear(in_features=16, out_features=8, bias=True)
  )
  (fc1): Linear(in_features=8, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (decoder): decoder(
    (lstm1): LSTM(8, 16, batch_first=True)
    (lstm2): LSTM(16, 32, batch_first=True)
    (lstm3): LSTM(32, 353, batch_first=True)
    (h_proj_1_to_2): Linear(in_features=16, out_features=32, bias=True)
    (c_proj_1_to_2): Linear(in_features=16, out_features=32, bias=True)
    (h_proj_2_to_3): Linear(in_features=32, out_features=353, bias=True)
    (c_proj_2_to_3): Linear(in_features=32, out_features=353,

In [ ]:
%%time
def train(output,data):
    model.train()
    for d in data:
        #print(type(d))
        x = d[0][0].to(device)
        y = d[1].to(device)
        optimizer.zero_grad()
        out = model(x)
        # print(out[0][9])
        output.append([out[0][train_window-1]])
        loss = criterion(out[0][train_window-1], y[0][0])
        loss.backward()
        optimizer.step()
    return float(loss) , output







############ results
########### result change layer 8 to 32
# Epoch: 01, Loss: 0.0188
# Epoch: 02, Loss: 0.0273
# Epoch: 03, Loss: 0.0033
# CPU times: user 6min 3s, sys: 3.62 s, total: 6min 6s
# Wall time: 6min 12s

########### result add hidden ayer 32
# Epoch: 01, Loss: 0.0021
# Epoch: 02, Loss: 0.0378
# Epoch: 03, Loss: 0.0069
# CPU times: user 7min 58s, sys: 4.28 s, total: 8min 2s
# Wall time: 8min 11s

########### result add hidden layer 32 to decoder too and add decoder to ruad model with gpu
# Epoch: 01, Loss: 0.0200
# Epoch: 02, Loss: 0.0210
# Epoch: 03, Loss: 0.0231
# CPU times: user 13min 7s, sys: 14.9 s, total: 13min 22s
# Wall time: 13min 5s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu
# Epoch: 01, Loss: 0.0089
# Epoch: 02, Loss: 0.0483
# Epoch: 03, Loss: 0.0035
# CPU times: user 12min 38s, sys: 13.5 s, total: 12min 51s
# Wall time: 12min 34s

########### result first ruad model with gpu and add timestamp column and window 10 and seq
# Epoch: 01, Loss: 0.0107
# Epoch: 02, Loss: 0.0098
# Epoch: 03, Loss: 0.0219
# CPU times: user 11min 17s, sys: 4.6 s, total: 11min 22s
# Wall time: 11min 28s

########### result first ruad model with gpu and add timestamp column and window 10 and remove seq
# Epoch: 01, Loss: 0.0252
# Epoch: 02, Loss: 0.0093
# Epoch: 03, Loss: 0.0188
# CPU times: user 11min 8s, sys: 4.67 s, total: 11min 12s
# Wall time: 11min 19s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , adam optimizer
# Epoch: 01, Loss: 0.0365
# Epoch: 02, Loss: 0.0473
# Epoch: 03, Loss: 0.0536
# CPU times: user 12min 14s, sys: 13.8 s, total: 12min 28s
# Wall time: 12min 12s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , SGD optimizer
# Epoch: 01, Loss: 0.0594
# Epoch: 02, Loss: 0.0260
# Epoch: 03, Loss: 0.0311
# CPU times: user 11min 27s, sys: 13.3 s, total: 11min 40s
# Wall time: 11min 24s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , SGD optimizer, epoch 8
# Epoch: 01, Loss: 0.0270
# Epoch: 02, Loss: 0.0267
# Epoch: 03, Loss: 0.0317
# Epoch: 04, Loss: 0.0313
# Epoch: 05, Loss: 0.0615
# CPU times: user 19min 21s, sys: 23.7 s, total: 19min 44s
# Wall time: 19min 18s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , SGD optimizer, epoch 5 , add h,c
# Epoch: 01, Loss: 0.0211
# Epoch: 02, Loss: 0.0153
# Epoch: 03, Loss: 0.0244
# Epoch: 04, Loss: 0.0031
# Epoch: 05, Loss: 0.0129
# CPU times: user 20min 12s, sys: 12 s, total: 20min 24s
# Wall time: 20min 39s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , SGD optimizer, epoch 6 , add h,c
# Epoch: 01, Loss: 0.0211
# Epoch: 02, Loss: 0.0153
# Epoch: 03, Loss: 0.0244
# Epoch: 04, Loss: 0.0031
# Epoch: 05, Loss: 0.0129
# Epoch: 06, Loss: 0.0030
# CPU times: user 24min 16s, sys: 14.37 s, total: 24min 30s
# Wall time: 24min 48s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq , SGD optimizer, epoch 6 , add h,c
# Epoch: 01, Loss: 0.0042
# Epoch: 02, Loss: 0.0035
# Epoch: 03, Loss: 0.0061
# Epoch: 04, Loss: 0.0053
# Epoch: 05, Loss: 0.0214
# Epoch: 06, Loss: 0.0109
# CPU times: user 26min 13s, sys: 14.2 s, total: 26min 27s
# Wall time: 26min 46s


CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 9.06 µs


In [ ]:
output=[]
i=0
for epoch in range(6):
    loss ,output = train(output,train_loader_value_0)
    if i!=5:
      output=[]
    i+=1
    print(f'Epoch: {epoch+1:02d}, Loss: {loss:.4f}')

# Epoch: 01, Loss: 0.0250
# Epoch: 02, Loss: 0.0022
# Epoch: 03, Loss: 0.0035
# Epoch: 04, Loss: 0.0222
# Epoch: 05, Loss: 0.0103
# Epoch: 06, Loss: 0.0174

Epoch: 01, Loss: 0.0226
Epoch: 02, Loss: 0.0051
Epoch: 03, Loss: 0.0024
Epoch: 04, Loss: 0.0250
Epoch: 05, Loss: 0.0096
Epoch: 06, Loss: 0.0182


In [ ]:
### before run this you need to rerun the model first
output=[]
i=0
for epoch in range(6):
    loss ,output = train(output,train_loader_value_1)
    if i!=5:
      output=[]
    i+=1
    print(f'Epoch: {epoch+1:02d}, Loss: {loss:.4f}')

# Epoch: 01, Loss: 0.3565
# Epoch: 02, Loss: 0.3171
# Epoch: 03, Loss: 0.2247
# Epoch: 04, Loss: 0.1146
# Epoch: 05, Loss: 0.0676
# Epoch: 06, Loss: 0.0487

Epoch: 01, Loss: 0.3560
Epoch: 02, Loss: 0.3177
Epoch: 03, Loss: 0.2342
Epoch: 04, Loss: 0.1247
Epoch: 05, Loss: 0.0718
Epoch: 06, Loss: 0.0499


In [ ]:
# def tensor_to_dataframe(tensor, columns, device):
#     numpy_array = tensor.cpu().numpy()  # Convert tensor to NumPy array
#     df = pd.DataFrame(numpy_array, columns=columns)  # Create DataFrame with original column names
#     return df

In [ ]:
### scale ,name run for both
df_list=[]
for l in output:
  ### tensore darim
  numpy_array = l[0].cpu().detach().numpy().reshape(1, -1)
  df=scaler.inverse_transform(numpy_array)
  df_list.append(df)


In [ ]:
df_combined_value_0 = pd.concat([pd.DataFrame(df, columns=names) for df in df_list], ignore_index=True)
df_combined_value_0=df_combined_value_0.astype(original_dtypes)

In [ ]:
df_combined_value_0.astype(original_dtypes)

,ambient_avg,ambient_std,ambient_min,ambient_max,dimm0_temp_avg,dimm0_temp_std,dimm0_temp_min,dimm0_temp_max,dimm10_temp_avg,dimm10_temp_std,...,ps1_output_curre_min,ps1_output_curre_max,ps1_output_volta_avg,ps1_output_volta_std,ps1_output_volta_min,ps1_output_volta_max,total_power_avg,total_power_std,total_power_min,total_power_max
0,19.111752,0.103980,19.992167,17.902309,26.270266,0.067379,27.112228,25.222149,26.528244,0.090421,...,18.275103,39.154720,11.900920,0.005312,11.893191,11.915222,577.942322,74.799545,420.272797,895.026855
1,19.117580,0.104262,19.990133,17.904127,26.272457,0.067526,27.116638,25.226612,26.532906,0.090639,...,18.280901,39.150028,11.901451,0.005411,11.893406,11.915354,577.976257,74.803375,420.340057,894.947021
2,19.117775,0.103951,19.996365,17.907173,26.277351,0.067495,27.117325,25.228928,26.532713,0.090369,...,18.284412,39.157024,11.901302,0.005636,11.893641,11.915649,578.005371,74.797600,420.395416,895.156738
3,19.121439,0.104116,19.991594,17.906439,26.277132,0.067671,27.119759,25.232410,26.536427,0.090832,...,18.288742,39.149418,11.901828,0.005610,11.893736,11.915611,578.036194,74.809120,420.432281,895.038269
4,19.121456,0.104174,19.993963,17.909210,26.279051,0.067690,27.120850,25.232677,26.536688,0.090741,...,18.284801,39.168396,11.901640,0.005354,11.893816,11.915809,577.963074,74.805000,420.440918,895.351074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50973,19.166368,0.108198,20.021065,17.940775,26.353802,0.070800,27.166538,25.318016,26.568689,0.090083,...,18.560356,39.332359,11.917655,0.011026,11.909896,11.932873,580.112976,74.277794,425.805939,899.159546
50974,19.168177,0.108540,20.022564,17.943552,26.355526,0.070913,27.169922,25.320120,26.572124,0.090596,...,18.555870,39.344448,11.917758,0.010801,11.909965,11.933003,580.006531,74.272469,425.821777,899.357117
50975,19.171080,0.108355,20.017975,17.939854,26.355747,0.070728,27.168447,25.318634,26.567858,0.090666,...,18.555767,39.329178,11.917792,0.010996,11.909823,11.932695,579.956055,74.283432,425.683014,899.087708
50976,19.172592,0.108151,20.022430,17.942844,26.361603,0.070868,27.170610,25.322227,26.567360,0.090759,...,18.556122,39.346024,11.917690,0.011035,11.910133,11.933034,580.020203,74.289375,425.713623,899.466797


In [ ]:
df_combined_value_1 = pd.concat([pd.DataFrame(df, columns=names) for df in df_list], ignore_index=True)
df_combined_value_1=df_combined_value_1.astype(original_dtypes)

In [ ]:
df_combined_value_1.astype(original_dtypes)

,ambient_avg,ambient_std,ambient_min,ambient_max,dimm0_temp_avg,dimm0_temp_std,dimm0_temp_min,dimm0_temp_max,dimm10_temp_avg,dimm10_temp_std,...,ps1_output_curre_min,ps1_output_curre_max,ps1_output_volta_avg,ps1_output_volta_std,ps1_output_volta_min,ps1_output_volta_max,total_power_avg,total_power_std,total_power_min,total_power_max
0,16.511742,0.537568,21.241655,16.609711,20.935974,0.067787,22.521778,19.300072,19.444149,-0.106271,...,17.394190,22.776196,7.970160,-1.343442,8.230649,6.827636,132.617447,53.575878,322.927795,171.825424
1,16.518711,0.538093,21.248766,16.618549,20.947105,0.067718,22.531761,19.310629,19.454264,-0.106461,...,17.419117,22.794359,7.973316,-1.343758,8.233836,6.831477,133.050522,53.630142,323.457733,172.303024
2,16.525967,0.538496,21.255892,16.627605,20.958941,0.067668,22.542450,19.321573,19.464586,-0.106664,...,17.444313,22.813217,7.976456,-1.344011,8.236936,6.835398,133.492523,53.678631,323.984741,172.770920
3,16.532103,0.538860,21.262745,16.635462,20.970705,0.067615,22.552967,19.331196,19.474541,-0.106832,...,17.458532,22.827965,7.979299,-1.344236,8.239762,6.838822,133.672714,53.801933,324.285858,173.066101
4,16.537916,0.539143,21.269560,16.643272,20.982231,0.067560,22.563198,19.341015,19.484224,-0.107003,...,17.472218,22.840908,7.982114,-1.344438,8.242537,6.842218,133.853424,53.956291,324.579468,173.411240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1286,19.209158,0.288275,22.540888,19.391027,23.213606,0.104860,23.744442,21.675346,22.019180,-0.015106,...,17.035469,22.796524,8.798644,-0.860561,8.847560,8.015622,253.902023,21.929222,351.090576,273.108551
1287,19.216629,0.289383,22.548180,19.396925,23.224052,0.105362,23.754021,21.681767,22.029116,-0.014207,...,17.050325,22.823280,8.801364,-0.861054,8.850632,8.019228,254.032394,22.246624,351.405853,273.834381
1288,19.221880,0.289831,22.553593,19.403145,23.232677,0.105302,23.762798,21.690315,22.036083,-0.014362,...,17.063000,22.841478,8.803512,-0.861314,8.852817,8.022197,254.189453,22.655771,351.649506,274.508087
1289,19.225655,0.290169,22.557562,19.407211,23.239725,0.105202,23.769672,21.696798,22.042427,-0.014492,...,17.070305,22.848766,8.805331,-0.861195,8.854741,8.024618,254.240387,22.845333,351.787872,274.668335


In [ ]:
df_combined_value_0[df_combined_value_0["ambient_avg"]< 0.30]

,ambient_avg,ambient_std,ambient_min,ambient_max,dimm0_temp_avg,dimm0_temp_std,dimm0_temp_min,dimm0_temp_max,dimm10_temp_avg,dimm10_temp_std,...,ps1_output_curre_min,ps1_output_curre_max,ps1_output_volta_avg,ps1_output_volta_std,ps1_output_volta_min,ps1_output_volta_max,total_power_avg,total_power_std,total_power_min,total_power_max
16089,0.299966,-0.017516,0.094556,0.141478,0.303372,0.071357,0.088286,-0.131195,-0.017650,0.103793,...,0.022426,-0.265496,-0.158172,0.045067,0.289073,0.351710,-0.077246,0.007189,0.216376,0.105203
16098,0.299797,-0.017483,0.094488,0.141496,0.303203,0.071329,0.088136,-0.131319,-0.017801,0.103698,...,0.022339,-0.265623,-0.158376,0.045118,0.288861,0.351559,-0.077382,0.007201,0.216317,0.104995
16128,0.299690,-0.017427,0.094476,0.141536,0.303130,0.071322,0.088146,-0.131331,-0.017822,0.103640,...,0.022356,-0.265510,-0.158361,0.045133,0.288834,0.351544,-0.077299,0.007214,0.216276,0.104990
16145,0.299548,-0.017421,0.094362,0.141549,0.302935,0.071275,0.087964,-0.131435,-0.017948,0.103552,...,0.022262,-0.265596,-0.158546,0.045218,0.288620,0.351402,-0.077435,0.007197,0.216236,0.104884
16204,0.299334,-0.017455,0.094220,0.141622,0.302739,0.071218,0.087788,-0.131511,-0.018064,0.103361,...,0.022238,-0.265714,-0.158740,0.045340,0.288355,0.351338,-0.077648,0.007197,0.216290,0.104699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50776,0.236689,-0.012383,0.077898,0.173877,0.249595,0.056697,0.063118,-0.120509,-0.026663,0.060799,...,0.020342,-0.239660,-0.151446,0.070896,0.240214,0.336183,-0.100342,0.023254,0.202247,0.064950
50798,0.236562,-0.012395,0.077797,0.173848,0.249486,0.056664,0.062934,-0.120632,-0.026806,0.060733,...,0.020252,-0.239771,-0.151653,0.070928,0.239993,0.336022,-0.100498,0.023283,0.202238,0.064827
50843,0.236552,-0.012367,0.077811,0.173851,0.249532,0.056704,0.063071,-0.120545,-0.026754,0.060780,...,0.020360,-0.239581,-0.151485,0.070917,0.240099,0.336120,-0.100404,0.023302,0.202214,0.064888
50858,0.236455,-0.012373,0.077746,0.173838,0.249441,0.056698,0.062965,-0.120607,-0.026850,0.060724,...,0.020334,-0.239656,-0.151614,0.070951,0.239949,0.336006,-0.100513,0.023304,0.202209,0.064790


In [ ]:
### value

df_combined_value_0['value'] = 0
df_combined_value_1['value'] = 1

<ipython-input-36-1e584eb0448a>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_combined_value_0['value'] = 0
<ipython-input-36-1e584eb0448a>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_combined_value_1['value'] = 1


In [ ]:
df_combined_value_1.head(5)

,ambient_avg,ambient_std,ambient_min,ambient_max,dimm0_temp_avg,dimm0_temp_std,dimm0_temp_min,dimm0_temp_max,dimm10_temp_avg,dimm10_temp_std,...,ps1_output_curre_max,ps1_output_volta_avg,ps1_output_volta_std,ps1_output_volta_min,ps1_output_volta_max,total_power_avg,total_power_std,total_power_min,total_power_max,value
0,16.511742,0.537568,21.241655,16.609711,20.935974,0.067787,22.521778,19.300072,19.444149,-0.106271,...,22.776196,7.970160,-1.343442,8.230649,6.827636,132.617447,53.575878,322.927795,171.825424,1
1,16.518711,0.538093,21.248766,16.618549,20.947105,0.067718,22.531761,19.310629,19.454264,-0.106461,...,22.794359,7.973316,-1.343758,8.233836,6.831477,133.050522,53.630142,323.457733,172.303024,1
2,16.525967,0.538496,21.255892,16.627605,20.958941,0.067668,22.542450,19.321573,19.464586,-0.106664,...,22.813217,7.976456,-1.344011,8.236936,6.835398,133.492523,53.678631,323.984741,172.770920,1
3,16.532103,0.538860,21.262745,16.635462,20.970705,0.067615,22.552967,19.331196,19.474541,-0.106832,...,22.827965,7.979299,-1.344236,8.239762,6.838822,133.672714,53.801933,324.285858,173.066101,1
4,16.537916,0.539143,21.269560,16.643272,20.982231,0.067560,22.563198,19.341015,19.484224,-0.107003,...,22.840908,7.982114,-1.344438,8.242537,6.842218,133.853424,53.956291,324.579468,173.411240,1


In [ ]:
df_combined_value_0.head(5)

,ambient_avg,ambient_std,ambient_min,ambient_max,dimm0_temp_avg,dimm0_temp_std,dimm0_temp_min,dimm0_temp_max,dimm10_temp_avg,dimm10_temp_std,...,ps1_output_curre_max,ps1_output_volta_avg,ps1_output_volta_std,ps1_output_volta_min,ps1_output_volta_max,total_power_avg,total_power_std,total_power_min,total_power_max,value
0,19.111752,0.103980,19.992167,17.902309,26.270266,0.067379,27.112228,25.222149,26.528244,0.090421,...,39.154720,11.900920,0.005312,11.893191,11.915222,577.942322,74.799545,420.272797,895.026855,0
1,19.117580,0.104262,19.990133,17.904127,26.272457,0.067526,27.116638,25.226612,26.532906,0.090639,...,39.150028,11.901451,0.005411,11.893406,11.915354,577.976257,74.803375,420.340057,894.947021,0
2,19.117775,0.103951,19.996365,17.907173,26.277351,0.067495,27.117325,25.228928,26.532713,0.090369,...,39.157024,11.901302,0.005636,11.893641,11.915649,578.005371,74.797600,420.395416,895.156738,0
3,19.121439,0.104116,19.991594,17.906439,26.277132,0.067671,27.119759,25.232410,26.536427,0.090832,...,39.149418,11.901828,0.005610,11.893736,11.915611,578.036194,74.809120,420.432281,895.038269,0
4,19.121456,0.104174,19.993963,17.909210,26.279051,0.067690,27.120850,25.232677,26.536688,0.090741,...,39.168396,11.901640,0.005354,11.893816,11.915809,577.963074,74.805000,420.440918,895.351074,0


In [ ]:
def combine_dataframes(df1, df2, pct_df1, pct_df2):
    """
    Combines an exact percentage of rows from two DataFrames into a new DataFrame.

    Parameters:
    - df1 (pd.DataFrame): The first DataFrame.
    - df2 (pd.DataFrame): The second DataFrame.
    - pct_df1 (float): Percentage of rows to take from df1 (between 0 and 1).
    - pct_df2 (float): Percentage of rows to take from df2 (between 0 and 1).

    Returns:
    - pd.DataFrame: A new DataFrame combining the specified percentages of rows from df1 and df2.
    """

    # Check if percentages are valid
    if not (0 <= pct_df1 <= 1) or not (0 <= pct_df2 <= 1):
        raise ValueError("Percentages must be between 0 and 1.")

    # Calculate the number of rows to sample from each DataFrame
    num_rows_df1 = int(len(df1) * pct_df1)
    num_rows_df2 = int(len(df2) * pct_df2)

    # Ensure we don't sample more rows than available
    if num_rows_df1 > len(df1):
        num_rows_df1 = len(df1)
    if num_rows_df2 > len(df2):
        num_rows_df2 = len(df2)

    # Sample the specified number of rows from each DataFrame
    df1_sample = df1.sample(n=num_rows_df1, random_state=1)  # random_state for reproducibility
    df2_sample = df2.sample(n=num_rows_df2, random_state=1)  # random_state for reproducibility

    # Combine the two sampled DataFrames
    combined_df = pd.concat([df1_sample, df2_sample], ignore_index=True)

    return combined_df

In [ ]:
result_df = combine_dataframes(df_combined_value_0, df_combined_value_1, pct_df1=0.5, pct_df2=0.3)

In [ ]:
result_df.head(5)

,ambient_avg,ambient_std,ambient_min,ambient_max,dimm0_temp_avg,dimm0_temp_std,dimm0_temp_min,dimm0_temp_max,dimm10_temp_avg,dimm10_temp_std,...,ps1_output_curre_max,ps1_output_volta_avg,ps1_output_volta_std,ps1_output_volta_min,ps1_output_volta_max,total_power_avg,total_power_std,total_power_min,total_power_max,value
0,19.148510,0.103844,20.000095,17.945583,26.307928,0.062935,27.138538,25.253262,26.522009,0.083560,...,38.680367,11.911870,0.011383,11.904340,11.926078,579.021545,73.882416,423.886566,889.121033,0
1,19.299364,0.105530,20.148287,18.053616,26.526463,0.065084,27.340868,25.499256,26.707712,0.084393,...,39.060268,11.927956,0.014339,11.921593,11.943518,584.083496,74.265717,427.878906,895.328735,0
2,19.160763,0.105061,20.002848,17.956367,26.347576,0.062218,27.185173,25.288696,26.591156,0.088762,...,38.891075,11.913747,0.007408,11.906398,11.927011,581.096802,74.496849,422.863892,895.610291,0
3,19.176367,0.104570,20.028547,17.966167,26.347809,0.060852,27.178061,25.303288,26.563274,0.086999,...,38.781696,11.914737,0.012257,11.906902,11.928796,581.259155,73.876663,426.541748,890.920288,0
4,19.040947,0.103504,19.903648,17.865681,26.226244,0.063633,27.073652,25.158014,26.487534,0.081357,...,38.910324,11.909749,0.003107,11.902328,11.924453,578.649414,74.136726,422.673401,893.596008,0


In [ ]:
shuffled_df = result_df.sample(frac=1, random_state=1).reset_index(drop=True)

In [ ]:
shuffled_df.head(5)

,ambient_avg,ambient_std,ambient_min,ambient_max,dimm0_temp_avg,dimm0_temp_std,dimm0_temp_min,dimm0_temp_max,dimm10_temp_avg,dimm10_temp_std,...,ps1_output_curre_max,ps1_output_volta_avg,ps1_output_volta_std,ps1_output_volta_min,ps1_output_volta_max,total_power_avg,total_power_std,total_power_min,total_power_max,value
0,19.229685,0.106754,20.110554,17.984207,26.407627,0.068458,27.195686,25.383928,26.604401,0.087480,...,39.293507,11.911378,0.010598,11.902648,11.925685,578.319885,74.955315,424.340240,894.078796,0
1,19.269480,0.106684,20.136211,18.044817,26.465017,0.065238,27.296246,25.433132,26.682890,0.087571,...,39.435295,11.917628,0.009728,11.909841,11.931731,581.546326,75.128845,425.029907,897.560120,0
2,19.221512,0.107285,20.095804,18.005213,26.405499,0.065453,27.233564,25.366463,26.637281,0.085510,...,39.193363,11.913942,0.008880,11.906287,11.928615,580.812134,74.832420,425.487579,895.730835,0
3,19.208990,0.106587,20.063225,17.995983,26.405542,0.064080,27.242531,25.357321,26.661263,0.095417,...,39.523842,11.917092,0.006943,11.909709,11.931262,583.762146,75.241547,424.703003,904.862366,0
4,19.305933,0.107999,20.159000,18.055079,26.513937,0.068207,27.317215,25.487492,26.694117,0.087581,...,39.413361,11.918992,0.013702,11.911250,11.933052,581.661743,74.943291,426.473907,896.086426,0


In [ ]:
### add time after ass anomaly for not being complicated on timestamp

timestamps = pd.date_range(start=pd.Timestamp('2021-04-02 00:45:00', tz='UTC'), periods=len(shuffled_df), freq='15T')

# Add timestamp and value columns to the DataFrame
shuffled_df['timestamp'] = timestamps

In [ ]:
shuffled_df

,ambient_avg,ambient_std,ambient_min,ambient_max,dimm0_temp_avg,dimm0_temp_std,dimm0_temp_min,dimm0_temp_max,dimm10_temp_avg,dimm10_temp_std,...,ps1_output_volta_avg,ps1_output_volta_std,ps1_output_volta_min,ps1_output_volta_max,total_power_avg,total_power_std,total_power_min,total_power_max,value,timestamp
0,19.229685,0.106754,20.110554,17.984207,26.407627,0.068458,27.195686,25.383928,26.604401,0.087480,...,11.911378,0.010598,11.902648,11.925685,578.319885,74.955315,424.340240,894.078796,0,2021-04-02 00:45:00+00:00
1,19.269480,0.106684,20.136211,18.044817,26.465017,0.065238,27.296246,25.433132,26.682890,0.087571,...,11.917628,0.009728,11.909841,11.931731,581.546326,75.128845,425.029907,897.560120,0,2021-04-02 01:00:00+00:00
2,19.221512,0.107285,20.095804,18.005213,26.405499,0.065453,27.233564,25.366463,26.637281,0.085510,...,11.913942,0.008880,11.906287,11.928615,580.812134,74.832420,425.487579,895.730835,0,2021-04-02 01:15:00+00:00
3,19.208990,0.106587,20.063225,17.995983,26.405542,0.064080,27.242531,25.357321,26.661263,0.095417,...,11.917092,0.006943,11.909709,11.931262,583.762146,75.241547,424.703003,904.862366,0,2021-04-02 01:30:00+00:00
4,19.305933,0.107999,20.159000,18.055079,26.513937,0.068207,27.317215,25.487492,26.694117,0.087581,...,11.918992,0.013702,11.911250,11.933052,581.661743,74.943291,426.473907,896.086426,0,2021-04-02 01:45:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25871,19.091082,0.104253,19.957909,17.892220,26.263477,0.061618,27.112799,25.216839,26.512440,0.089997,...,11.910684,0.004730,11.903470,11.926427,580.288208,74.253952,421.351898,896.867126,0,2021-12-27 12:30:00+00:00
25872,19.173651,0.107651,20.062759,17.967918,26.348282,0.065629,27.181952,25.307352,26.602537,0.088499,...,11.914075,0.005035,11.905065,11.928512,584.218079,74.940369,425.206573,894.466064,0,2021-12-27 12:45:00+00:00
25873,19.214636,0.106983,20.047539,18.001740,26.421118,0.063836,27.243435,25.365341,26.638086,0.093569,...,11.917341,0.009962,11.909878,11.930631,582.999268,74.679390,424.577698,898.002747,0,2021-12-27 13:00:00+00:00
25874,19.121593,0.105935,20.012653,17.923569,26.289528,0.065904,27.142294,25.244379,26.570311,0.085342,...,11.906970,0.002526,11.898543,11.921307,577.907104,74.876015,421.552002,892.608093,0,2021-12-27 13:15:00+00:00


In [ ]:
shuffled_df[shuffled_df["value"]==1]

,ambient_avg,ambient_std,ambient_min,ambient_max,dimm0_temp_avg,dimm0_temp_std,dimm0_temp_min,dimm0_temp_max,dimm10_temp_avg,dimm10_temp_std,...,ps1_output_volta_avg,ps1_output_volta_std,ps1_output_volta_min,ps1_output_volta_max,total_power_avg,total_power_std,total_power_min,total_power_max,value,timestamp
141,19.270132,0.473648,23.459457,19.843128,25.402191,0.096072,26.028769,23.458006,23.823317,-0.072404,...,9.225692,-1.125304,9.350658,8.487459,245.405273,51.775448,406.902985,286.419403,1,2021-04-03 12:00:00+00:00
145,19.211500,0.310327,22.708858,19.465525,23.704622,0.102138,24.238541,22.090672,22.447853,-0.026738,...,8.901213,-0.909045,8.962700,8.134380,252.816360,24.074251,361.962524,273.930756,1,2021-04-03 13:00:00+00:00
170,18.351419,0.497250,22.832832,18.854355,24.076813,0.081935,25.001352,22.142399,22.445147,-0.096724,...,8.810247,-1.240547,8.983029,7.922435,209.977997,58.478729,380.808472,250.907349,1,2021-04-03 19:15:00+00:00
333,17.220572,0.572842,21.995470,17.498827,22.224354,0.065064,23.664562,20.427414,20.577345,-0.121085,...,8.296931,-1.363898,8.545703,7.224230,155.796768,69.538254,354.686584,211.769455,1,2021-04-05 12:00:00+00:00
397,19.356390,0.468385,23.520210,19.931705,25.522938,0.095185,26.129639,23.581299,23.956932,-0.070479,...,9.263024,-1.114642,9.383281,8.536812,247.488434,52.021130,408.387451,288.466766,1,2021-04-06 04:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25575,18.357164,0.498774,22.844540,18.863968,24.100039,0.081848,25.024508,22.162371,22.465939,-0.097143,...,8.815665,-1.241995,8.988734,7.928722,209.966110,58.740635,381.338135,251.071121,1,2021-12-24 10:30:00+00:00
25602,18.457611,0.492155,22.880724,18.964735,24.222729,0.087390,25.105900,22.292419,22.618427,-0.093453,...,8.870728,-1.219014,9.035807,8.012244,214.941116,55.490341,383.378448,254.223434,1,2021-12-24 17:15:00+00:00
25694,18.965719,0.502133,23.290127,19.531601,25.027807,0.089513,25.769117,23.055552,23.397964,-0.086517,...,9.101373,-1.184458,9.249625,8.318508,239.057709,56.389229,404.881256,279.876556,1,2021-12-25 16:15:00+00:00
25732,18.048607,0.575531,22.824039,18.531866,23.630190,0.068666,24.837452,21.681711,21.922651,-0.120800,...,8.660631,-1.343084,8.882943,7.683593,170.794647,67.196922,376.610535,220.888412,1,2021-12-26 01:45:00+00:00


In [ ]:
### save it on google drive
### save it on google drive
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/MasterProject/'+'create_data_my_model_seprate_anomaly_0'+'.parquet'
shuffled_df.to_parquet(file_path, index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#0.8765328998395648
#0.8723729179726375    result of change hidden layer from 8 to 32
#0.9065247187071686    result of add one layer 32 between encoder :)
#0.904416749009233     result add hidden layer 32 to decoder too and add decoder to ruad model with gpu (this is pretty good)
#0.8655186444515994    result first ruad model with gpu and add timestamp column and woindow 10 and seq , adam optimizer
#0.8967381497429511    result first ruad model with gpu and add timestamp column and woindow 10 and remove seq , adam optimizer
#0.3060959342747257    result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq, adam optimizer
#0.34766214604908874   result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq, SGD optimizer
#0.36209899126422795   result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq, SGD optimizer , epoch 5 +3 (i forgot to reset)
#0.907213292243243     result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq, SGD optimizer , epoch 5 , add , h,c
#0.9098181020482538    result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq, SGD optimizer , epoch 6 , add , h,c
#0.9069957536027086    result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq, SGD optimizer , epoch 6 , add , h,c